# Shared Task: Critical thinking generation
## Loadind Data


In [ ]:
pip install -q social_agents/

In [ ]:
pip show social_agents

In [ ]:
import tqdm
# Reading the data
import json
import pandas as pd
import social_agents

from social_agents.utils import get_st_data


for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    break


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

## Prerequisites
env, LLM, ...

In [1]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"


##  1. Zero Shot LLM to start with o3-mini

In [ ]:

from social_agents.graph_tools import BasicCQModel
model_name = "o3-mini"

basic_agent = BasicCQModel(llm_name = model_name, temperature=None ) #interrupt_before=['human_feedback'], checkpointer=memory)


display(Image(basic_agent.graph.get_graph(xray=1).draw_mermaid_png()))

# RUN
# basic_agent.run_experiment()



### Evaulate 

- Run shell 

```shell

python3 eval_scripts/evaluation.py \
    --metric similarity \
    --input_path data_splits/validation.json \
    --submission_path output/output_o3-mini_temperatureNA.json \
    --threshold 0.6 

```

- OUTPUT : output/output_o3-mini_temperatureNA._eval_similarity_06.json



**Overall count**

| **Questions Labels** |  **#**  | **ratio** |
|:--------------------|-------:|---------:|
| useful               | **329** |  **0,59** |
| unhelpful            |      63 |      0,11 |
| Invalid              |       8 |      0,01 |
| Not able to evaluate |     158 |      0,28 |
| **Total**            | **558** |         1 |


**Overall count within each argument**

| **n/3 useful questions per arg** | **# of arguments** | **ratio** |
|:-------------------------------:|-------------------:|:---------:|
|                               0/3 |                 17 |      0,10 |
|                               1/3 |                 51 |      0,27 |
|                               2/3 |             **76** |      0,40 |
|                               3/3 |                 42 |      0,23 |
| **Total**                       |            **186** |         1 |

## Social Agents

In [2]:
import pandas as pd

EXPERIMENT_SETTINGS_FILE_PATH = "output/elbaff_experiment/experiment_settings.csv"
experiment_settings = pd.read_csv(EXPERIMENT_SETTINGS_FILE_PATH)


# exp_row = experiment_settings.iloc[20].to_dict()
# exp_row

In [ ]:
import ast
from social_agents import objects
from social_agents.graph_tools import  CQSTAbstractAgent, SocialAgentBuilder
import ast
from tqdm import tqdm

llm_name = "gpt-4o-mini-2024-07-18"# "o3-mini-2025-01-31"
temperature = 0.7
for idx, row in tqdm(experiment_settings.iterrows(), total=len(experiment_settings)):
    if "easy_going" in list(ast.literal_eval(row["traits"])):
        social_agent = SocialAgentBuilder(
            model_thread_id=row["thread_id"],
            llm_name = llm_name,
            llm_num = row["number_of_agents"],
            experiment_name= row["experiment_name"].format(llm_name= llm_name),
            temperature=temperature,
            collaborative_strategy=list(ast.literal_eval(row["strategies"])),
            agent_trait_lst=list(ast.literal_eval(row["traits"])))
        social_agent.run_experiment(data_type = "validation", save= True)
        print(f'finished {row["experiment_name"].format(llm_name= llm_name)}')




In [ ]:
from IPython.display import Image, display
display(Image(social_agent.graph.get_graph(xray=1).draw_mermaid_png()))

[1,2].extend([6])